Building Gen Ai apps with llm's
LLm's are trained on a data for ex The training data for GPT-4o is limited to data from October 2023 or earlier.
but to get info after 2023 with GPT 4-o requires tools with which our LLM interacts


In [ ]:
import os

#Arxiv-Research Paper
#Tools creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper
from langchain_core.prompts import MessagesPlaceholder
from tenacity import retry_if_exception_message

In [ ]:
wiki_api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki_tool=WikipediaQueryRun(api_wrapper=wiki_api_wrapper)
wiki_tool.name

In [ ]:
arxiv_api_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv_tool=ArxivQueryRun(api_wrapper=arxiv_api_wrapper)
arxiv_tool.name

In [ ]:
tools=[wiki_tool,arxiv_tool]

In [ ]:
#Custom Tool (RAG Tool)
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings,ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["GOOGLE_API_KEY"]=os.getenv("GEMINI_KEY")

In [ ]:
docs=WebBaseLoader("https://docs.smith.langchain.com/").load()
splitted=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50).split_documents(docs)
db=FAISS.from_documents(splitted,GoogleGenerativeAIEmbeddings(model="models/text-embedding-004"))
retriever=db.as_retriever()
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [ ]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,name="langsmith_search",description="search any information on langsmith")
retriever_tool

In [ ]:
tools=[arxiv_tool,wiki_tool,retriever_tool]
tools

hub.pull = “get a reusable chain/prompt/agent template from the Hub ”.

it's like github where we can use someone's else code so we don't have to makesomething from scratch

 Inside, it’s basically:

A prompt template that tells your LLM how to act like an agent that can call functions/tools.

It includes instructions, formatting, and placeholders for:

The system message (“You are a helpful assistant that can use tools…”)

The user message (what the user asked)

Function call JSON structure (so the LLM knows how to format the function/tool call)

In [ ]:
## Prompt Template
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

now for llm to use tool we need agents

In [ ]:
#Agent
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

In [ ]:
#Agent Executor
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)#to see the working verbose=True
agent_executor

In [ ]:
agent_executor.invoke({"input":"tell me something about Sachin Tendulkar"})
##this invokes 3 agent executor chain because we gave 3 tools wikipedia arxiv and retriever (for langsmith website)

In [ ]:
agent_executor.invoke({"input":"tell me something about langsmith"})

In [ ]:
agent_executor.invoke({"input": "What's the paper 1706.03762 about?"})